# Training
-----------------------------------

Here we're training on a pretty large dataset of infected and uninfected images. To download the dataset yourself, you can go [here](https://www.kaggle.com/iarunava/cell-images-for-detecting-malaria/download)

## Setting up the dataset

Follow the steps below to get your dataset set up.
 - Extract the downloaded zip
 - Rename `cell_images` to `train_cell_images`
 - Create a directory called `model`. This is where the trained model will be saved.

## Importing libraries and config

In [11]:
import os
import cv2
import keras
import numpy as np
import sklearn
import tensorflow as tf
from PIL import Image
from keras.layers import SeparableConv2D, MaxPooling2D, Dense, Dropout, GlobalMaxPooling2D
from keras.models import Sequential
from tensorflow.keras.models import load_model
from keras.callbacks import EarlyStopping, TensorBoard
from sklearn.model_selection import train_test_split

physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  pass

## Loading data

In [ ]:
data = []
labels = []

# Load infected images
Parasitized = os.listdir("./dataset/train_cell_images/Parasitized/")
for p in Parasitized:
    try:
        image = cv2.imread("./dataset/train_cell_images/Parasitized/" + p)
        image_from_array = Image.fromarray(image, 'RGB')
        size_image = image_from_array.resize((50, 50))
        rotated45 = size_image.rotate(45)
        rotated75 = size_image.rotate(75)
        blur = cv2.blur(np.array(size_image), (10, 10))
        data.append(np.array(size_image))
        data.append(np.array(rotated45))
        data.append(np.array(rotated75))
        data.append(np.array(blur))
        labels.append(0.0)
        labels.append(0.0)
        labels.append(0.0)
        labels.append(0.0)
    except AttributeError:
        pass

# Load uninfected images
Uninfected = os.listdir("./dataset/train_cell_images/Uninfected/")
for u in Uninfected:
    try:
        image = cv2.imread("./dataset/train_cell_images/Uninfected/" + u)
        image_from_array = Image.fromarray(image, 'RGB')
        size_image = image_from_array.resize((50, 50))
        rotated45 = size_image.rotate(45)
        rotated75 = size_image.rotate(75)
        data.append(np.array(size_image))
        data.append(np.array(rotated45))
        data.append(np.array(rotated75))
        labels.append(1.0)
        labels.append(1.0)
        labels.append(1.0)
    except AttributeError:
        pass

# Convert image pixels to numpy arrays for easy processing
cells = np.array(data)
labels = np.array(labels)

np.save("model/cells", cells)
np.save("model/labels", labels)

## Quick load preprocessed data

In [2]:
cells = np.load("model/cells.npy")
labels = np.load("model/labels.npy")

## Preprocessing data

In [3]:
# Shuffle cells to prevent some sort of bias
s = np.arange(cells.shape[0])
np.random.shuffle(s)
cells = cells[s]
labels = labels[s]

num_classes = len(np.unique(labels))
len_data = len(cells)

# Split into train and test datasets
x_train, x_test, y_train, y_test = train_test_split(cells, labels)

x_train = x_train.astype('float32') / 255  # Normalize RGB values by dividing with 255
x_test = x_test.astype('float32') / 255
train_len = len(x_train)
test_len = len(x_test)

print(x_train.shape)
print(y_train.shape)

(72339, 50, 50, 3)
(72339,)


## Create and save model

In [7]:
callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        min_delta=1e-2,
        patience=10,
        verbose=1),
    TensorBoard(
        log_dir='logs',
        histogram_freq=1,
        write_graph=True,
        write_images=True
    )

]

# Create a sequential keras model
model = Sequential()
model.add(SeparableConv2D(filters=8, kernel_size=3, padding="same", activation="relu", input_shape=(50, 50, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(SeparableConv2D(filters=16, kernel_size=3, padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(SeparableConv2D(filters=32, kernel_size=3, padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalMaxPooling2D())
model.add(Dense(32, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(16, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))
model.summary()

# compile the model with loss function as binary_crossentropy and using adam optimizer you can test result by trying
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit (train) the model. Using a batch size which is x^2 optimizes training on my GPU
model.fit(x_train, y_train, batch_size=512, epochs=20, verbose=1, callbacks=callbacks, validation_data=(x_test, y_test))

accuracy = model.evaluate(x_test, y_test, verbose=1)
print('\n', 'Validation accuracy: ', accuracy[1])

model.save('model/saved_model.pb')

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d_4 (Separabl (None, 50, 50, 8)         59        
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 25, 25, 8)         0         
_________________________________________________________________
separable_conv2d_5 (Separabl (None, 25, 25, 16)        216       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 12, 12, 16)        0         
_________________________________________________________________
separable_conv2d_6 (Separabl (None, 12, 12, 32)        688       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 6, 6, 32)         

# Prediction
------------------

## Setting up prediction dataset
 - Create a directory called `test_cell_images`
 - Copy any two images of your choosing into this directory. Name them `test1` and `test2`.

## Predicting the type of cell

In [18]:
from tensorflow.keras.models import load_model

def convert_to_array(img):
    im = cv2.imread(img)
    img_ = Image.fromarray(im, 'RGB')
    image = img_.resize((50, 50))
    image = np.array(image)
    image = image / 255
    return np.array([image], dtype=np.float32)


def get_cell_name(label):
    if label == 0:
        return "Infected"
    if label == 1:
        return "Uninfected"


def predict_cell(file):
    model = load_model('./model/saved_model.pb')
    arr = convert_to_array(file)
    score = model.predict(arr, verbose=3)
    cell = get_cell_name(round(score[0][0]))
    return "The predicted cell is an " + cell + " cell."


print(predict_cell('./dataset/test_cell_images/test1.png')) # Infected
print(predict_cell('./dataset/test_cell_images/test2.png')) # Not infected


The predicted cell is an Infected cell.
The predicted cell is an Uninfected cell.


# Deployment
-----------------------------------------------

Often, we'll want to deploy a model to a mobile device, or we just want to reduce its size. TFLite is a great way to reduce size and make models compatible with mobile devices, without losing much performance and accuracy.

## Conversion to TFLite

In [12]:
# Load keras model from disk
model = load_model('./model/saved_model.pb')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

# Convert the model
tflite_model = converter.convert()

# Create the tflite model file
with open('model/cells.tflite', "wb") as f:
    f.write(tflite_model)

## Run inference on tflite model

In [28]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="model/cells.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_img = convert_to_array('./dataset/test_cell_images/test2.png')

# Run inference
interpreter.set_tensor(input_details[0]['index'], input_img)
interpreter.invoke()
output = round(interpreter.get_tensor(output_details[0]['index'])[0][0])
print("The predicted cell is an " + get_cell_name(output) + " cell.")

The predicted cell is an Uninfected cell.
